# Regresion Lineal y Logistica

En este ejemplo intentaremos adivinar cuantos likes tiene un video de tiktok a partir de la cantidad de vistas

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# tiktok vistas y likes de distintos videos

views = [234_000, 8_000, 217_000, 25_000, 62_000, 107_000, 
         140_000, 191_000, 189_000, 496_000, 221_000, 238_000]
likes = [ 39_000,   362,  31_000,  1_500,  4_000,   5_000,   
           6_000,  15_000,  11_000,  30_000,  30_000,  31_000]

In [ ]:
df = pd.DataFrame({"Views":views,"Likes":likes})

In [ ]:
df.head()

In [ ]:
sns.relplot(x="Views",y="Likes", data=df);

## Regresion Lineal

Suponemos que la cantidad de likes es linealmente proporcional a la cantidad de vistas. Por lo tanto buscaremos cual es la funcion que dibuja una linea cuya suma de las distancias a los puntos de nuestra tabla de entrenamiento es la menor posible. Esta linea se puede obtener de forma analitica: https://en.wikipedia.org/wiki/Simple_linear_regression

In [ ]:
df["x - xmean"] = df["Views"] - df["Views"].mean()
df["y - ymean"] = df["Likes"] - df["Likes"].mean()
df["sq(x - xmean)"] = df["x - xmean"] * df["x - xmean"]

In [ ]:
df.head()

Obtengo la pendiente de la linea

In [ ]:
w = np.sum(df["x - xmean"] * df["y - ymean"]) / df["sq(x - xmean)"].sum()

In [ ]:
w

Obtengo la interseccion con el eje y (ordenada al origen)

In [ ]:
b = np.mean(df["Likes"] - w * df["Views"])

In [ ]:
b

In [ ]:
linea = lambda x: w * x + b

In [ ]:
coef = np.polyfit(df["Views"],df["Likes"],1)
linea = np.poly1d(coef)
print(coef)
print(linea)

In [ ]:
sns.relplot(x="Views",y="Likes", data=df)
plt.plot(df["Views"], linea(df["Views"]), '-');

## Usando la libreria Scikit Learn

La libreria sklearn nos permite obtener esta linea de forma mas simple: 

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
from sklearn.linear_model import LinearRegression
X = df.loc[:,["Views"]]
y = df["Likes"]
reg = LinearRegression().fit(X, y)

Obtengo la pendiente de la linea

In [ ]:
reg.coef_

Obtengo la interseccion con el eje y (ordenada al origen)

In [ ]:
reg.intercept_

# IA: adivinar y luego minimzar el error repetidamente

Otra forma de obtener dicha linea es a traves del algoritmo de gradiente descendiente. Este algorimo es muy importante en inteligencia artificial ya que nos permite encontrar un minimo de forma iterativa.

### Generamos lineas al azar

Empezamos generando lineas al azar para ver que tan buenas predictoras son. Para ello calculamos la suma de las distancias de los puntos de la tabla de entrenamiento a las distintas rectas.

In [ ]:
#inicializo con valores aleatorios
b = 0.0
w = 0.1
#hago mi prediccion
def prediccion(x, w, b): 
    return w * x + b
#mi suma de los errores al cuadrado sera
def costo(x, w, b, y): 
    return np.sum((y - prediccion(x,w,b))**2)

Solo elegiremos la pendiente w al azar, la ordenada al origen b la dejaremos fija en cero para que el ejemplo sea mas simple:

In [ ]:
sns.relplot(x="Views",y="Likes", data=df)
pendientes = [-0.05,0.0,0.05,0.1,0.15,0.2]
costos = []
for w in pendientes:
    costos.append(costo(df["Views"], w, b, df["Likes"]))
    plt.plot(df["Views"], prediccion(df["Views"], w, b), '-');

### Calculamos los errores de cada linea que creamos

Elegido un w al azar podemos calcular su costo, es decir, la suma de diferencias al cuadrado de los puntos de la tabla de entrenamiento a la linea dada por w. 

In [ ]:
#tabla de la suma de errores para cada linea
df_costos = pd.DataFrame({"w":pendientes,"costo":costos})
df_costos

Esta tabla nos da el costo de cada linea dependiendo de la pendiente w elegida. Si extendemos esta tabla para todos los w obtenemos la llamada funcion de costo:

In [ ]:
sns.relplot(x="w",y="costo", data=df_costos);
plt.plot(pendientes, costos, '-');

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
xnew = np.linspace(np.min(pendientes), np.max(pendientes), 50) 
spl = make_interp_spline(pendientes, costos, k=3)
plt.plot(xnew, spl(xnew));

## El truco matematico del gradiente descendiente

Teniendo la funcion de costo podemos utilizar la pendiente (derivada) de dicha funcion de costo para saber en que direccion tenemos que buscar para encontrar el minimo:

In [ ]:
#utilizamos la libreria sympy para obtener la derivada de la funcion de costo
import sympy as sy

In [ ]:
x,y,w,b = sy.symbols('x y w b')
costFunf = (y-(w*x+b))**2
costFunf

Derivada parcial de la funcion de costo respecto de w:

In [ ]:
sy.diff(costFunf,w)

Derivada parcial de la funcion de costo respecto de b:

In [ ]:
sy.diff(costFunf,b)

In [ ]:
del x,y,w,b

Las siguientes funciones nos indican entonces en que direccion tenemos que ir para encontrar el minimo:

In [ ]:
def gradW(x,y,w,b):
    grad = -2*x*(-b-w*x+y)
    grad = np.sum(grad)
    return grad

In [ ]:
def gradb(x,y,w,b):
    grad = 2*b+2*w*x-2*y
    grad = np.sum(grad)
    return grad

### El algortimo del gradiente descendiente

A continuacion empiezo con un w al azar y luego lo voy modificando de a poco para ir en direccion hacia el minimo costo. Tengo que tener cuidado de ir modificando de a poco porque si el paso que doy es demasiado grande el algoritmo puede diverger (no encontrara el minimo) o puede tardar mucho si el paso es muy pequeño. A el tamaño de cada paso hacia el minimo se lo conoce como Learning Rate (LR)

In [ ]:
x = np.array(df["Views"])
y = np.array(df["Likes"])
w = 0.2
b = 0.0
ws, cs = [], []
for i in range(20):
    ws.append(w)
    cost = costo(x, w, b, y)
    cs.append(cost)
    grad = gradW(x, y, w, b)
    print(f"w:{w:0.3f} cost:{int(cost):10}")
    w = w - 0.0000000000003 * grad #LR adecuado. El algoritmo encuentra el minimo
    #w = w - 0.0000000000018 * grad #LR demasiado grande. El algoritmo diverge
    #w = w - 0.00000000000001 * grad #LR demasiado chico. El algoritmo tarda mucho

In [ ]:
from scipy.interpolate import make_interp_spline, BSpline
xnew = np.linspace(np.min(pendientes), np.max(pendientes), 50) 
spl = make_interp_spline(pendientes, costos, k=3)
plt.plot(xnew, spl(xnew));
plt.plot(ws, cs, 'x-');

## Extendemos el algoritmo para mas dimensiones

Aqui extendemos el algoritmo para encontrar tambien la ordenada al origen. De la misma forma podemos extenderlo para mas dimensiones.

In [ ]:
x = np.array(df["Views"])
y = np.array(df["Likes"])
w = 0.2
b = 0.0
for epoch in range(60):
    cost = costo(x, w, b, y)
    tmp_gradW = gradW(x, y, w, b)
    tmp_gradb = gradb(x, y, w, b)
    if epoch%5 == 0: print(f"w:{w:0.4f} b:{int(b):4} cost:{int(cost):10}")
    w = w - 0.000000000001 * tmp_gradW
    b = b - 0.03 * tmp_gradb

# Regresion Logistica

Utilizando la tecnica del gradiente descendiente en este ejemplo intentaremos adivinar si a una persona que le damos un prestamo nos devolvera el dinero. En este caso nuestros valores a predecir no son numeros sino una variable categorica que puede tener los valores SI o NO.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame({"Salario":[1500,600,1800,700,1400,3100,2900,4200,5100,4500],
                  "Ahorro": [11000,5700,1500,7100,4900,11000,13000,14000,18000,16000],
                  "Devolvio":[0,0,0,0,0,1,1,1,1,1]})

In [ ]:
df.head()

In [ ]:
sns.relplot(x="Salario", y="Ahorro", hue="Devolvio", data=df);

Debido a que la variable a predecir es categorica, la funcion que buscaremos no sera una linea recta si no una funcion sigmoidea. En donde todos los valores que son mayores a cierto umbral seran iguales a SI y los menores a ese umbral seran iguales a NO:

In [ ]:
sigmoid = lambda x: 1 / (1 + np.exp(-x))

In [ ]:
plt.plot(np.linspace(-10,10,100), sigmoid(np.linspace(-10,10,100)), '-');

In [ ]:
sns.relplot(x="Salario", y="Devolvio", hue="Devolvio", data=df);
plt.plot(np.linspace(0,6000,100), sigmoid(5*(np.linspace(0,6000,100)/1000-2.5)), '-');

Tambien cambiara la funcion para medir que tan bien mi funcion sigmoidea se ajusta a la tabla de entrenamiento. Por lo tanto no usaremos la funcion de suma de diferencias de cuadrados sino que utilizaremos la funcion llamada entropia cruzada:

In [ ]:
costoy1 = lambda x: -np.log(sigmoid(x))
plt.plot(np.linspace(0,1,100), costoy1(np.linspace(0,5,100)), '-');
costoy0 = lambda x: -np.log(1-sigmoid(x))
plt.plot(np.linspace(0,1,100), costoy0(np.linspace(-5,0,100)), '-');

Con estas dos modificaciones podemos volver a utilizar el algoritmo de gradiente descendiente para encontrar la funcion que mejor se adapta a mi tabla de entrenamiento.

## Con Scikit Learn

In [ ]:
from sklearn.linear_model import LogisticRegression
X = df.loc[:,["Salario","Ahorro"]].to_numpy()
y = df.loc[:,"Devolvio"].to_numpy()
clf = LogisticRegression(random_state=0).fit(X, y)

In [ ]:
clf.predict([[3_000,15_000]])

In [ ]:
clf.predict([[900,8_000]])

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)